In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import socket
import sys
import requests
import requests_oauthlib
import json
import pandas as pd
from collections import Counter
import re

# Credentials for 
ACCESS_TOKEN = '187939782-zmGcY8Uu7nzcEAdmvchA3PrzAU9LPiW9rqYkSvLG'
ACCESS_SECRET = 'BW3eHcY52cIw26qqcmTr8wQihpHWLKNU89zHUOPgI69nd'
CONSUMER_KEY = 'I6DLUgf9LJIfgSWvcXX2KQcGV'
CONSUMER_SECRET = '8dUzdOiKXnxPxVjW5Lk28iq7WkrYSuZPg2YcJDhfNsGeJGQYjh'
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)


def send_tweets_to_spark(conn,http_resp):
    for line in http_resp.iter_lines():
        try:
            #conn, addr = new_socket.accept()
            full_tweet = json.loads(line)
            # refer https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html for details
            tweet_text = str(full_tweet["created_at"])+"*////*"+str(full_tweet["coordinates"]["coordinates"][0])+"*////*"+str(full_tweet["coordinates"]["coordinates"][1])+"*////*"+full_tweet["place"]["name"]+"*////*"+full_tweet["text"]
            #tweet_text  = full_tweet['text']
            #row = [ user, text, latitude, longitude, gender ]
            data = tweet_text+'\n'
            #save_tweet(tweet_text)
            #print("Tweet Text: " + tweet_text)
            #print ("------------------------------------------")
            conn.send(data.encode('ascii','ignore'))
            #conn.close()
        except:
            e = sys.exc_info()[0]
            #print("Error: %s" % e)


def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-180,-60,180,60'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response


TCP_IP = "localhost"
TCP_PORT = 5000
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected to :",addr)
resp = get_tweets()
print("Starting getting tweets.")
send_tweets_to_spark(conn,resp)